<a href="https://colab.research.google.com/github/Sangwonsdsd/KNN-Numpy/blob/master/homework_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework
 - Backbone, localization header, classification header 를 설계해 아래 코드가 작동될 수 있도록 구현해주세요. <br> 
 - 데이터의 크기가 변동되었음으로 해당 변동사항을 확인해주세요.
 - train ㅎ시 overfeat loss 가 최소 10 이하로 떨어져야 합니다. 
 - 수업 코드에서 제공되었던 mnist_local_generator.py 와 utils.py 을 업로드 해 사용해야 합니다. 
 

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.regularizers import l2
from mnist_local_generator import mnist_localization_generator
import matplotlib.pyplot as plt 
import numpy as np 
from utils import xywh2xyxy, draw_rectangle
import cv2
import tensorflow.keras.backend as K
import tensorflow as tf 
from tensorflow.keras.losses import CategoricalCrossentropy, MSE

In [ ]:
# load mnist dataset
# Generate mnist data for localization
(train_images, train_cls_true, train_reg_true), (test_images, test_cls_true, test_reg_true) = \
    mnist_localization_generator((128, 128), (128, 128),
                                 background=True, 
                                 image_size_range=(60, 80),
                                 image_ratio_range=(0.5, 1.5),
                                 n_sample=10000)

print(train_images.shape, test_images.shape)

## Backbone 
 - 아래 Cell Convolution 을 활용한 backbone 코드를 작성해 주세요

In [ ]:
K.clear_session()
# 아래 코드에 Convolution 을 활용한 backbone 코드를 작성해 주세요

## Localization header
 - 아래 Cell에 Convolution 을 활용한 localization 예측 코드를 작성해 주세요

In [ ]:
# 아래 코드에 Convolution 을 활용한 localization header 파트 코드를 작성해 주세요

## Classification header
 - 아래 Cell에 Convolution 을 활용한 localization 예측 코드를 작성해 주세요

In [ ]:
# 아래 코드에 Convolution 을 활용한 classification header 파트 코드를 작성해 주세요

## Concatenate header

In [ ]:
output = Concatenate(axis=-1)([loc_output, cls_output])

## Define overfeat loss 

In [ ]:
def overfeat_mse(true, pred):
    """
    :param true: ndarray, 4d tensor (NHWC) 단 C=4
    :param pred: ndarray or tensor, 4d tensor (NHWC), 단 C=4
    :return: mse_, float,
    """

    # slicing classification and regression
    mse = MSE(true, pred)
    mse_ = tf.math.reduce_mean(mse)

    return mse_

In [ ]:
def overfeat_cee(true, pred):
    """
    :param true: ndarray, 4d tensor (NHWC) 단 C=11
    :param pred: ndarray or tensor, 4d tensor (NHWC), 단 C=11
    :return: cee_, float,
    """

    cee = CategoricalCrossentropy()
    cee_ = cee(true, pred)

    return cee_ 

In [ ]:
def overfeat_loss(true, pred):
    """
    Description:
    :param true: ndarray, shape = (N, 1, 1, 15=(4+11))
    :param pred: ndarray, shape = (N, 1, 1, 15=(4+11))
    """

    true_reg = true[:, :, :, :4]
    true_cls = true[:, :, :, 4:]

    pred_reg = pred[:, :, :, :4]
    pred_cls = pred[:, :, :, 4:]

    # positive 인 data의 loss 만 localization loss 에 추가 
    pos_mask = true_cls[:, :, :, -1] != 1   
    pos_true_reg = true_reg[pos_mask]
    pos_pred_reg = pred_reg[pos_mask]

    mse_loss = overfeat_mse(pos_true_reg, pos_pred_reg)
    cee_loss = overfeat_cee(true_cls, pred_cls)

    total_loss = mse_loss*0.01 + cee_loss*2
    return total_loss


In [ ]:
def metric_mse(true, pred):
    """
    Description:
    :param true: ndarray, shape = (N, 1, 1, 15=(4+11))
    :param pred: ndarray, shape = (N, 1, 1, 15=(4+11))
    """

    true_reg = true[:, :, :, :4]
    true_cls = true[:, :, :, 4:]
    pred_cls = pred[:, :, :, 4:]
    pred_reg = pred[:, :, :, :4]

    # positive 인 data의 loss 만 localization loss 에 추가 
    pos_mask = true_cls[:, :, :, -1] != 1   
    pos_true_reg = true_reg[pos_mask]
    pos_pred_reg = pred_reg[pos_mask]

    mse_loss = overfeat_mse(pos_true_reg, pos_pred_reg)
    return mse_loss


In [ ]:
def metric_cee(true, pred):
    """
    Description:
    :param true: ndarray, shape = (N, 1, 1, 15=(4+11))
    :param pred: ndarray, shape = (N, 1, 1, 15=(4+11))
    """
    true_cls = true[:, :, :, 4:]
    pred_cls = pred[:, :, :, 4:]
    cee_loss = overfeat_cee(true_cls, pred_cls)

    return cee_loss


## Compile model

In [ ]:
# generate keras model
model = Model(input_, output)
model.compile(optimizer='adam', loss=overfeat_loss, metrics=[metric_mse, metric_cee])
model.summary()

## Train model

In [ ]:
model.fit(train_images, train_reg_cls, batch_size=32, epochs=100)

## Predict Model

In [ ]:
pred_loc_cls = model.predict(test_images[:10])

## Visualization

In [ ]:
pred_loc = pred_loc_cls[:, ..., :4]
pred_cls = pred_loc_cls[:, ..., 4:]

In [ ]:
pred_loc = np.squeeze(pred_loc)
pred_cls = np.argmax(pred_cls, axis=-1)
pred_cls = np.squeeze(pred_cls)

In [ ]:
pred_loc = xywh2xyxy(pred_loc)

In [ ]:
index = 5
rected_image = draw_rectangle(test_images[index, ..., 0], pred_loc[index])
plt.imshow(rected_image)
plt.title(pred_cls[index])

In [ ]:
model.save('./best_model.h5')